<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [17]:
data = dict({'x1': [0, 0, 1, 0, 1, 1, 0], 
             'x2': [0, 1, 0, 1, 0, 1, 0], 
             'x3': [1, 1, 1, 0, 0, 1, 0], 
             'y': [0, 1, 1, 1, 1, 0, 0]
            })

base_data = pd.DataFrame.from_dict(data)

In [18]:
base_data.head()

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1


In [19]:
# Get feature matrix / target vector.
features = list(base_data)[:-1]
target = list(base_data)[-1:]

features

['x1', 'x2', 'x3']

In [32]:
import numpy as np
import pandas as pd
from scipy.special import expit

# Define neural network class object.
class first_neural_network(object):
    def __init__(self, inputs=3, hidden=4, output=1):
        """ Construct object """
        self.inputs = inputs
        self.hiddenNodes = hidden
        self.outputNodes = output

        # Initial Weights
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)

    def sigmoid(self, s):
        """ Squish input via sigmoid function """
        return expit(s)

    def sigmoid_derivative(self, s):
        """ Get derivative (slope) of squished sigmoid """
        return s * (1 - s)

    def fit(self, X, y, n_iter=10):
        """ Fit and train NN on X matrix input """
        self.n_iter = n_iter
        for iterations in range(n_iter):
            # Feed Forward  
            self.hidden_sum = np.dot(X, self.weights1)
            self.activated_hidden = self.sigmoid(self.hidden_sum)
            self.output_sum = np.dot(self.activated_hidden, self.weights2)
            self.activated_output = self.sigmoid(self.output_sum)

            # Get error deltas and tune weights -> Backprop
            self.out_error = y - self.activated_output
            self.dx_out = (self.out_error * 
                           self.sigmoid_derivative(self.activated_output))
            self.stage1_error = np.dot(self.dx_out, self.weights2.T)
            self.dx_stage1 = (self.stage1_error * 
                              self.sigmoid_derivative(self.activated_hidden))
            self.weights1 += np.dot(X.T, self.dx_stage1)
            self.weights2 += np.dot(self.activated_hidden.T, self.dx_out)
        return {'Model' : 'MultiLayerPerceptron', 'Iterations':self.n_iter}
            
    def predict(self, X):
        self.hidden_sum = np.dot(X, self.weights1)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        self.output_sum = self.activated_hidden.dot(self.weights2)
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output

In [33]:
# Instantiate, fit and create predictions from the model.
model = first_neural_network(inputs=3, hidden=4, output=1)
model.fit(base_data[features], base_data[target], n_iter=1000)
model.predict(base_data[features])

array([[0.0075039 ],
       [0.95281296],
       [0.95663252],
       [0.95394713],
       [0.95062472],
       [0.05690697],
       [0.08102575]])

In [34]:
prediction = [1 if x > 0.5 else 0 for x in model.predict(base_data[features])]
print(prediction, [x for x in base_data['y']])

[0, 1, 1, 1, 1, 0, 0] [0, 1, 1, 1, 1, 0, 0]


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [35]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [36]:
# input image dimensions
img_rows, img_cols = 28, 28

In [37]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [38]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [39]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [40]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [41]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [42]:
from sklearn.metrics import accuracy_score

model = first_neural_network(inputs=784, hidden=15, output=1)
model.fit(x_train, y_train, n_iter=1000)
y_pred = model.predict(x_test)



accuracy_score(y_test, y_pred)

MemoryError: 

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?